In [6]:
# Import libs
import praw
from tqdm import tqdm
import json
import pickle
import re

In [ ]:
# Initializating reddit account for parsing
# Private info changed to ####
reddit = praw.Reddit(
    client_id="####",
    client_secret="####",
    user_agent="####",
    ratelimit_seconds = 4
)

subreddit = reddit.subreddit('casualconversation')

# Function to preprocess texts
def string_fix(s):
    return s.replace("\n"," ").replace("\r"," ").replace('"','\\"')

In [ ]:
# Loading files
with open("id_list_to_parse", "rb") as f:
    id_list_to_parse = pickle.load(f)

with open("already_parsed_ids.json", 'r', encoding = "utf-8") as f:
    already_parsed_ids = json.load(f)

posts = open("posts.json", 'a', encoding = "utf-8")
comments = open("comments.json", 'a', encoding = "utf-8")


# Parsing process
for post_id in tqdm(id_list_to_parse):
    post = reddit.submission(id=post_id)

    # Making sure we have not parsed the post yet (by id)
    if post.id not in already_parsed_ids["ids"]:
            
        # Adding the post to the dictionaty of visited posts
        already_parsed_ids["ids"].append(post.id)
        
        # Filling the line with the needed data about the post
        line_post = '{' + '\"author\": \"{0}\", '\
        '\"author_flair_css_class\": \"{1}\", '\
        '\"author_flair_text\": \"{2}\", '\
        '\"created_utc\": \"{3}\", '\
        '\"domain\": \"{4}\", '\
        '\"full_link\": \"{5}\", '\
        '\"gilded\": \"{6}\", '\
        '\"id\": \"{7}\", '\
        '\"is_self\": \"{8}\", '\
        '\"media_embed\": \"{9}\", '\
        '\"num_comments\": \"{10}\", '\
        '\"over_18\": \"{11}\", '\
        '\"permalink\": \"{12}\", '\
        '\"retrieved_on\": \"{13}\", '\
        '\"score\": \"{14}\", '\
        '\"secure_media_embed\": \"{15}\", '\
        '\"selftext\": \"{16}\", '\
        '\"stickied\": \"{17}\", '\
        '\"subreddit\": \"{18}\", '\
        '\"subreddit_id\": \"{19}\", '\
        '\"thumbnail\": \"{20}\", '\
        '\"title\": \"{21}\", '\
        '\"url\": \"{22}\"'\
        .format(
        post.author,
        post.author_flair_css_class,
        post.author_flair_text,
        post.created_utc,
        post.domain,
        post.full_link,
        post.gilded,
        post.id,
        post.is_self,
        post.media_embed,
        post.num_comments,
        post.over_18,
        post.permalink,
        post.retrieved_on,
        post.score,
        post.secure_media_embed,
        re.sub(r'[^\w\s]', '', str(post.selftext)),
        post.stickied,
        post.subreddit,
        post.subreddit_id,
        post.thumbnail,
        re.sub(r'[^\w\s]', '', str(post.title)),
        post.url
        ) + '}\n'

        # Appending that line to a file
        posts.write(line_post)

        # Collect all comments
        post.comments.replace_more(limit=None)
        comms = post.comments.list()
        for comm in comms:
            line_comment = '{' + '\"author\": \"{0}\", '\
            '\"author_created_utc\": \"{1}\", '\
            '\"author_flair_css_class\": \"{2}\", '\
            '\"author_flair_text\": \"{3}\", '\
            '\"author_fullname\": \"{4}\", '\
            '\"body\": \"{5}\", '\
            '\"controversiality\": \"{6}\", '\
            '\"distinguished\": \"{7}\", '\
            '\"gilded\": \"{8}\", '\
            '\"id\": \"{9}\", '\
            '\"link_id\": \"{10}\", '\
            '\"nest_level\": \"{11}\", '\
            '\"parent_id\": \"{12}\", '\
            '\"retrieved_on\": {13}, '\
            '\"score\": {14}, '\
            '\"score_hidden\": \"{15}\", '\
            '\"subreddit\": \"{16}\", '\
            '\"subreddit_id\": \"{17}\"'\
            .format(
            comm.author,
            comm.author_created_utc,
            comm.author_flair_css_class,
            comm.author_flair_text,
            comm.author_fullname,
            re.sub(r'[^\w\s]', '', str(comm.body)),
            comm.controversiality,
            comm.distinguished,
            comm.gilded,
            comm.id,
            comm.link_id,
            comm.nest_level,
            comm.parent_id,
            comm.retrieved_on,
            comm.score,
            comm.score_hidden,
            comm.subreddit,
            comm.subreddit_id
            ) + '}\n'
            
            # Appending that line to a file
            comments.write(line_comment)

In [ ]:
# Saving results
with open("id_list.json", 'w', encoding = "utf-8") as f:
    json.dump(already_parsed_ids, f)

posts.close()
comments.close()